In [8]:
# %% [code]
# Instalación de dependencias necesarias
!pip install transformers diffusers moviepy gradio elevenlabs
!pip install git+https://github.com/openai/whisper.git


import torch
from diffusers import StableDiffusionXLPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips
import whisper
import gradio as gr

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-rlbwm41h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-rlbwm41h
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


ImportError: cannot import name 'ElevenLabsClient' from 'elevenlabs' (/usr/local/lib/python3.11/dist-packages/elevenlabs/__init__.py)

In [ ]:
# 1. Generación de guion con un modelo GPT-2 en español
from transformers import AutoModelForCausalLM, AutoTokenizer

modelo_nombre = "datificate/gpt2-small-spanish"  # Modelo gratuito en español
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
model = AutoModelForCausalLM.from_pretrained(modelo_nombre)

def generar_guion(tema, max_length=200):
    prompt = f"Escribe un guion explicativo sobre {tema} en tono educativo."
    # Si el tokenizer no tiene definido un token de padding, lo asignamos al token de fin de secuencia
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    # Tokenizamos incluyendo el attention_mask y habilitando el padding
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        do_sample=True,
        temperature=0.7
    )
    guion = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return guion


def generar_guion_de_tema(tema):
    return generar_guion(tema)

# 2. Conversión de guion a voz con ElevenLabs
import elevenlabs


def generar_audio(texto, archivo_salida="audio.mp3"):


  from elevenlabs import play
  from elevenlabs.client import ElevenLabs
  client = ElevenLabs(api_key="TU KEY") # No te olvides de tu key de ElevenLabs


  audio = client.generate(
        text=texto,
        voice="Rachel",
        model="eleven_multilingual_v2"
    )


  with open(archivo_salida, "wb") as f:
        audio_bytes = b"".join(audio)
        f.write(audio_bytes)

  play(audio)

  return archivo_salida



# 3. Generación de imagen con Stable Diffusion
"""
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.to("cuda")

def generar_imagen(prompt, archivo_salida="imagen.png"):
    imagen = pipe(f"Ilustración digital sin texto de: {prompt}", guidance_scale=7.5).images[0]
    imagen.save(archivo_salida)
    return archivo_salida
"""
import torch
from diffusers import StableDiffusionXLPipeline

# Carga del modelo SDXL
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16
)
pipe.to("cuda")

def generar_imagen(prompt, archivo_salida="imagen.png"):
    # Puedes ajustar el prompt según tus necesidades
    imagen = pipe(f"Ilustración digital sin texto de: {prompt}", guidance_scale=7.5).images[0]
    imagen.save(archivo_salida)
    return archivo_salida


# 4. Edición y ensamblaje del video con MoviePy
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips

def crear_video(imagenes, audio, salida="video.mp4", duracion_por_imagen=5):
    clips = [ImageClip(img).set_duration(duracion_por_imagen) for img in imagenes]
    video = concatenate_videoclips(clips, method="compose")
    audio_clip = AudioFileClip(audio)
    video = video.set_audio(audio_clip)
    video.write_videofile(salida, fps=24)
    return salida

# 5. Generación de subtítulos automáticos con Whisper
import whisper

model_whisper = whisper.load_model("small")

def generar_subtitulos(audio_file, subtitulo_salida="subtitulos.srt"):
    resultado = model_whisper.transcribe(audio_file)
    with open(subtitulo_salida, "w", encoding="utf-8") as f:
        f.write(resultado["text"])
    return subtitulo_salida

# 6. Función global para generar video a partir del guion editado
def generar_video_con_guion(guion_editado):
    # Genera el audio a partir del guion
    archivo_audio = generar_audio(guion_editado)
    # Separa el guion en oraciones (suponiendo que cada punto indica una parte)
    oraciones = [oracion.strip() for oracion in guion_editado.split('.') if oracion.strip()]
    duracion_audio = AudioFileClip(archivo_audio).duration
    duracion_por_imagen = duracion_audio / len(oraciones) if oraciones else 5

    imagenes = []
    for i, oracion in enumerate(oraciones):
        archivo_imagen = f"imagen_{i}.png"
        generar_imagen(oracion, archivo_salida=archivo_imagen)
        imagenes.append(archivo_imagen)

    archivo_video = crear_video(imagenes, archivo_audio, duracion_por_imagen=duracion_por_imagen)
    archivo_subtitulos = generar_subtitulos(archivo_audio)

    return archivo_video, archivo_subtitulos, guion_editado

# 7. Interfaz final con Gradio (dos pasos: generar/editar guion y generar video)
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Generador Automático de Videos Explicativos")
    gr.Markdown("Primero, ingresa un tema para generar el guion. Luego, edítalo si lo deseas y genera el video final.")

    with gr.Tab("Generar y Editar Guion"):
        with gr.Row():
            tema_input = gr.Textbox(label="Tema del Video", placeholder="Ej: La historia del internet")
            btn_guion = gr.Button("Generar Guion")
        guion_text = gr.Textbox(label="Guion Generado (editable)", lines=10)
        btn_guion.click(fn=generar_guion_de_tema, inputs=tema_input, outputs=guion_text)

    with gr.Tab("Generar Video"):
        guion_editable = gr.Textbox(label="Guion Editable", lines=10)
        btn_video = gr.Button("Generar Video")
        video_output = gr.File(label="Video Explicativo")
        subtitles_output = gr.File(label="Subtítulos (.srt)")
        guion_final = gr.Textbox(label="Guion Final")
        btn_video.click(fn=generar_video_con_guion, inputs=guion_editable, outputs=[video_output, subtitles_output, guion_final])

demo.launch()


input("Presiona Enter para finalizar...")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  checkpoint = torch.load(fp, map_location=device)



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1059666dfa7d0033ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Moviepy - Building video video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
Presiona Enter para finalizar...


''